In [1]:
!pip install wandb -qU

In [2]:
import pickle
import pandas as pd
import random
import json
import wandb
import time
import collections
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

wandb.login(key="aec6fef7ba56ee445129472eb583718b8e529934")

wandb: Currently logged in as: sentimovie. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cp /content/drive/MyDrive/MLSD/proj/phase2/reviews/train.pkl .
!cp /content/drive/MyDrive/MLSD/proj/phase2/reviews/val.pkl .

In [5]:
with open('train.pkl', 'rb') as f:
    train_set = pickle.load(f)
    
with open('val.pkl', 'rb') as f:
    val_set = pickle.load(f)

In [6]:
train = pd.DataFrame.from_dict(train_set)
val =  pd.DataFrame.from_dict(val_set)

train = train.dropna()
val = val.dropna()

X_train = train["x"]
X_val = val["x"]
Y_train = train["y"]
Y_val = val["y"]

In [7]:
def calculate_metric(Y_val, y_pred, phase):
    metrics_SVM = classification_report(Y_val, y_pred, output_dict=True)

    performance = {f"{phase}_acc": metrics_SVM["accuracy"],
                  f"{phase}_prec": metrics_SVM["weighted avg"]['precision'],
                  f"{phase}_rec": metrics_SVM["weighted avg"]['recall'],
                  f"{phase}_f1": metrics_SVM["weighted avg"]['f1-score']}
    return performance

# Feature Extraction

feature extraction with tf-idf



In [8]:
print("TFIDF Vectorizer")
vectorizer = TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_val = vectorizer.transform(X_val)

TFIDF Vectorizer


# Random

## Simple Random

In [27]:
run = wandb.init(project="baseline", name="Simple-Random", tags=['Random'])

label = ["Fresh","Rotten"]

y_pred = []
for i in range(len(Y_train)):
    y_pred.append(random.choice(label))
performance = calculate_metric(Y_train, y_pred, phase="train")
wandb.log(performance)

y_pred = []
for i in range(len(Y_val)):
    y_pred.append(random.choice(label))
performance = calculate_metric(Y_val, y_pred, phase="val")
wandb.log(performance)
wandb.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


train_acc,▁
train_f1,▁
train_prec,▁
train_rec,▁
val_acc,▁
val_f1,▁
val_prec,▁
val_rec,▁
train_acc,0.50021
train_f1,0.51019
train_prec,0.53928


## Weighted Random

We made the assumption that there is an equal probability for every class. Let's use the train split to figure out what the true probability is.

In [21]:
run = wandb.init(project="baseline", name="Weighted-Random", tags=['Random'])

p=[]
for i in label:
    p.append(collections.Counter(Y_train)[i]/len(Y_train))

y_pred = []
for i in range(len(Y_train)):
    y_pred.append(random.choices(label, weights=p ,k=1))
performance = calculate_metric(Y_train, y_pred, phase="train")
wandb.log(performance)

y_pred = []
for i in range(len(Y_val)):
    y_pred.append(random.choices(label, weights=p ,k=1))
performance = calculate_metric(Y_val, y_pred, phase="val")
wandb.log(performance)

wandb.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


train_acc,▁
train_f1,▁
train_prec,▁
train_rec,▁
val_acc,▁
val_f1,▁
val_prec,▁
val_rec,▁
train_acc,0.53987
train_f1,0.53991
train_prec,0.53995


# SVM

## Linear SVM

In [13]:
ex_num = 0

def run_model(config=None):
    global ex_num
    ex_num += 1
    with wandb.init(project="baseline", name=f"Linear-SVM-hp{ex_num}", 
                    tags=['Linear', 'svm'], config=config):
        ss = time.time()
        clf = SVC(C=wandb.config.c, kernel='linear', random_state=42, max_iter=100, verbose=2)
        clf.fit(tf_x_train, train['y'])
        e = time.time()
        print(f"Running time: {(e-ss):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_train)
        performance = calculate_metric(Y_train, y_pred, phase="train")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_val)
        performance = calculate_metric(Y_val, y_pred, phase="val")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        print(f"Total running time: {(e-ss):.3f}")


def main():
    sweep_config = {'method': 'grid'}

    metric = {'name': 'val_acc', 
            'goal': 'maximize'}

    sweep_config['metric'] = metric

    hyper_parameters = {
        'c': {'values': [0.01, 0.1, 1, 10, 100]},
    }
    sweep_config['parameters'] = hyper_parameters
    sweep_id = wandb.sweep(sweep_config, project="baseline")
    wandb.agent(sweep_id, run_model)

In [14]:
main()

Create sweep with ID: i5a5oyyz
Sweep URL: https://wandb.ai/sentimovie/baseline/sweeps/i5a5oyyz


wandb: Agent Starting Run: victxk2z with config:
wandb: 	c: 0.01
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[LibSVM]

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Running time: 17.737
Running time: 65.412
Running time: 8.137
Total running time: 91.287


...

[LibSVM]

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Running time: 13.584
Running time: 83.602
Running time: 5.050
Total running time: 102.238


train_acc,▁
train_f1,▁
train_prec,▁
train_rec,▁
val_acc,▁
val_f1,▁
val_prec,▁
val_rec,▁
train_acc,0.44449
train_f1,0.40727
train_prec,0.58507


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## RBF SVM

In [8]:
ex_num = 0

def run_model(config=None):
    global ex_num
    ex_num += 1
    with wandb.init(project="baseline", name=f"RBF-SVM-hp{ex_num}", 
                    tags=['RBF', 'svm'], config=config):
        s = time.time()
        clf = SVC(C=wandb.config.c, kernel='rbf', gamma=float(wandb.config.gamma),
                  random_state=42, max_iter=100, verbose=2)
        clf.fit(tf_x_train, train['y'])
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_train)
        performance = calculate_metric(Y_train, y_pred, phase="train")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_val)
        performance = calculate_metric(Y_val, y_pred, phase="val")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")


def main():
    sweep_config = {'method': 'grid'}

    metric = {'name': 'val_acc', 
            'goal': 'maximize'}

    sweep_config['metric'] = metric

    hyper_parameters = {
        'c': {'values': [0.1, 1, 10]},
        'gamma': {'values': [0, 1, 3, 9]},
    }
    sweep_config['parameters'] = hyper_parameters
    sweep_id = wandb.sweep(sweep_config, project="baseline")
    wandb.agent(sweep_id, run_model)

In [9]:
main()

Create sweep with ID: cj8w67wc
Sweep URL: https://wandb.ai/sentimovie/baseline/sweeps/cj8w67wc


wandb: Agent Starting Run: o9f2omt7 with config:
wandb: 	c: 0.1
wandb: 	gamma: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[LibSVM]

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Running time: 30.325


...

[LibSVM]

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Running time: 19.986
Running time: 71.767
Running time: 5.564


train_acc,▁
train_f1,▁
train_prec,▁
train_rec,▁
val_acc,▁
val_f1,▁
val_prec,▁
val_rec,▁
train_acc,0.36049
train_f1,0.19155
train_prec,0.72889


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Poly SVM

In [15]:
ex_num = 0

def run_model(config=None):
    global ex_num
    ex_num += 1
    with wandb.init(project="baseline", name=f"Poly-SVM-hp{ex_num}", 
                    tags=['Poly', 'svm'], config=config):
        ss = time.time()
        clf = SVC(C=wandb.config.c, kernel='poly', degree=wandb.config.degree, 
                  random_state=42, max_iter=100, verbose=1)
        clf.fit(tf_x_train, train['y'])
        e = time.time()
        print(f"Running time: {(e-ss):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_train)
        performance = calculate_metric(Y_train, y_pred, phase="train")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_val)
        performance = calculate_metric(Y_val, y_pred, phase="val")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        print(f"Total running time: {(e-ss):.3f}")


def main():
    sweep_config = {'method': 'grid'}

    metric = {'name': 'val_acc', 
            'goal': 'maximize'}

    sweep_config['metric'] = metric

    hyper_parameters = {
        'c': {'values': [0.1, 1, 10]},
        'degree': {'values': [2, 3, 4, 5]},
    }
    sweep_config['parameters'] = hyper_parameters
    sweep_id = wandb.sweep(sweep_config, project="baseline")
    wandb.agent(sweep_id, run_model)

In [16]:
main()

Create sweep with ID: x0m7vfn8
Sweep URL: https://wandb.ai/sentimovie/baseline/sweeps/x0m7vfn8


wandb: Agent Starting Run: hcjmkhpz with config:
wandb: 	c: 0.1
wandb: 	degree: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[LibSVM]

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Running time: 18.985
Running time: 75.306
Running time: 7.070
Total running time: 101.364


...

[LibSVM]

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Running time: 14.919
Running time: 61.816
Running time: 7.740
Total running time: 84.477


train_acc,▁
train_f1,▁
train_prec,▁
train_rec,▁
val_acc,▁
val_f1,▁
val_prec,▁
val_rec,▁
train_acc,0.36026
train_f1,0.19105
train_prec,0.68176


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# Logistic Regression

In [18]:
ex_num = 0

def run_model(config=None):
    global ex_num
    ex_num += 1
    with wandb.init(project="baseline", name=f"Logistic-Regression-hp{ex_num}", 
                    tags=['logistic'], config=config):
        ss = time.time()
        clf = LogisticRegression(solver="saga", C=wandb.config.c, random_state=42, 
                                 max_iter=100, verbose=2)
        clf.fit(tf_x_train, train['y'])
        e = time.time()
        print(f"Running time: {(e-ss):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_train)
        performance = calculate_metric(Y_train, y_pred, phase="train")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_val)
        performance = calculate_metric(Y_val, y_pred, phase="val")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        print(f"Total running time: {(e-ss):.3f}")


def main():
    sweep_config = {'method': 'grid'}

    metric = {'name': 'val_acc', 
            'goal': 'maximize'}

    sweep_config['metric'] = metric

    hyper_parameters = {
        'c': {'values': [0.01, 0.1, 1, 10, 100]},
    }
    sweep_config['parameters'] = hyper_parameters
    sweep_id = wandb.sweep(sweep_config, project="baseline")
    wandb.agent(sweep_id, run_model)

In [19]:
main()

Create sweep with ID: dgf722j1
Sweep URL: https://wandb.ai/sentimovie/baseline/sweeps/dgf722j1


wandb: Agent Starting Run: w40tmrv4 with config:
wandb: 	c: 0.01
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
rescaling...
convergence after 22 epochs took 18 seconds
Running time: 20.717


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.2s finished


...

max_iter reached after 82 seconds
Running time: 84.392


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min finished


Running time: 57.237
Running time: 4.224
Total running time: 145.855


train_acc,▁
train_f1,▁
train_prec,▁
train_rec,▁
val_acc,▁
val_f1,▁
val_prec,▁
val_rec,▁
train_acc,0.85575
train_f1,0.85454
train_prec,0.85448


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# Random Forest

In [7]:
ex_num = 0

def run_model(config=None):
    global ex_num
    ex_num += 1
    with wandb.init(project="baseline", name=f"RandomForestClassifier-hp{ex_num}", 
                    tags=['RandomForest', 'decision-tree'], config=config):
        ss = time.time()
        clf = RandomForestClassifier(n_estimators=10, 
                                         max_depth=int(wandb.config.max_depth), 
                                         random_state=42, verbose=2)
        clf.fit(tf_x_train, train['y'])
        e = time.time()
        print(f"Running time: {(e-ss):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_train)
        performance = calculate_metric(Y_train, y_pred, phase="train")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_val)
        performance = calculate_metric(Y_val, y_pred, phase="val")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        print(f"Total running time: {(e-ss):.3f}")


def main():
    sweep_config = {'method': 'grid'}

    metric = {'name': 'val_acc', 
            'goal': 'maximize'}

    sweep_config['metric'] = metric

    hyper_parameters = {
        'max_depth': {'values': [1, 10, 100, 1000]},
    }
    sweep_config['parameters'] = hyper_parameters
    sweep_id = wandb.sweep(sweep_config, project="baseline")
    wandb.agent(sweep_id, run_model)

In [8]:
main()

Create sweep with ID: dgts8wgh
Sweep URL: https://wandb.ai/sentimovie/baseline/sweeps/dgts8wgh


wandb: Agent Starting Run: qbvlm8is with config:
wandb: 	max_depth: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    1.2s finished


...

Running time: 80.473


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.9s finished


Running time: 9.365
Total running time: 1769.105


train_acc,▁
train_f1,▁
train_prec,▁
train_rec,▁
val_acc,▁
val_f1,▁
val_prec,▁
val_rec,▁
train_acc,0.97186
train_f1,0.97164
train_prec,0.97263


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# xgboost

In [10]:
ex_num = 0

def run_model(config=None):
    global ex_num
    ex_num += 1
    with wandb.init(project="baseline", name=f"xgboost-hp{ex_num}", 
                    tags=['xgboost'], config=config):
        ss = time.time()
        clf = GradientBoostingClassifier(n_estimators=10,
                                             learning_rate=wandb.config.lr, 
                                             random_state=42, 
                                             max_depth=int(wandb.config.max_depth), 
                                             verbose=2)
        clf.fit(tf_x_train, train['y'])
        e = time.time()
        print(f"Running time: {(e-ss):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_train)
        performance = calculate_metric(Y_train, y_pred, phase="train")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        s = time.time()
        y_pred = clf.predict(tf_x_val)
        performance = calculate_metric(Y_val, y_pred, phase="val")
        wandb.log(performance)
        e = time.time()
        print(f"Running time: {(e-s):.3f}")
        print(f"Total running time: {(e-ss):.3f}")


def main():
    sweep_config = {'method': 'grid'}

    metric = {'name': 'val_acc', 
            'goal': 'maximize'}

    sweep_config['metric'] = metric

    hyper_parameters = {
        'max_depth': {'values': [5, 10, 30]},
        'lr': {'values': [1.0, 0.1, 0.01]},
    }
    sweep_config['parameters'] = hyper_parameters
    sweep_id = wandb.sweep(sweep_config, project="baseline")
    wandb.agent(sweep_id, run_model)

In [11]:
main()

Create sweep with ID: a5ivv1rq
Sweep URL: https://wandb.ai/sentimovie/baseline/sweeps/a5ivv1rq


wandb: Agent Starting Run: k2071nlh with config:
wandb: 	lr: 1
wandb: 	max_depth: 5
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


      Iter       Train Loss   Remaining Time 
         1           1.2829            3.12m
         2           1.2653            2.58m
         3           1.2515            2.21m
         4           1.2389            1.84m
         5           1.2275            1.50m
         6           1.2180            1.19m
         7           1.2093           53.47s
         8           1.2008           35.39s
         9           1.1921           17.58s
        10           1.1852            0.00s
Running time: 176.731
Running time: 56.431
Running time: 4.550
Total running time: 237.715


train_acc,▁
train_f1,▁
train_prec,▁
train_rec,▁
val_acc,▁
val_f1,▁
val_prec,▁
val_rec,▁
train_acc,0.69074
train_f1,0.64378
train_prec,0.68991


wandb: Agent Starting Run: ix1pfwuw with config:
wandb: 	lr: 1
wandb: 	max_depth: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Running time: 6.264
Total running time: 1773.291


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train_acc,▁
train_f1,▁
train_prec,▁
train_rec,▁
val_acc,▁
val_f1,▁
val_prec,▁
val_rec,▁
train_acc,0.63988
train_f1,0.49936
train_prec,0.40945


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
